## Classifiers

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

classifierは原資産と時系列データからカテゴリ変数を出力する関数です。

```
F(asset, timestamp) -> category
```

classifierの例として取引所のIDを取り上げます。`morningstar.share_class_reference.exchange_id`と銘柄のインスタンスを用います。

In [2]:
from quantopian.pipeline.data import morningstar

# Since the underlying data of morningstar.share_class_reference.exchange_id
# is of type string, .latest returns a Classifier
exchange = morningstar.share_class_reference.exchange_id.latest

これまではFactorのインスタンスからlatest属性を参照していましたが、今回はclassifierを用いた文字列のデータを扱います。

銘柄に対してのモーニングスターのセクタコード算出します。元となるデータ型はint型ですがカテゴリ変数であるため数値ではありません。セクタコードを取得する倍には `Sector` クラスを用います。

In [3]:
from quantopian.pipeline.classifiers.morningstar import Sector
morningstar_sector = Sector()

上記は `morningstar.asset_classification.morningstar_sector_code.latest` と等価です。

### Building Filters from Classifiers

[Classifiers](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier) は `isnull`、`eq`、`startswith` などのメソッドが使えます。

たとえば、ニューヨーク証券取引所のセクタを選択する場合は、`exchange` classifierの `eq` メソッドを用います。

In [4]:
nyse_filter = exchange.eq('NYS')

このfilterは、最新の `exchange_id` から 'NYS'である銘柄に対して `True` を返します。

### Quantiles

ClassifiersはさまざまなFactorのメソッドから作成することができます。最も一般的な例として `quantiles` メソッドがあります。このメソッドはビン数を引数として渡します。下記の例は10日間の売買代金を十分位にしています。

In [5]:
a = AverageDollarVolume(window_length=10)

In [6]:
dollar_volume_decile = AverageDollarVolume(window_length=10).deciles()
top_decile = (dollar_volume_decile.eq(9))

piplineを実行するとこのようになります。

In [7]:
def make_pipeline():

    exchange = morningstar.share_class_reference.exchange_id.latest
    nyse_filter = exchange.eq('NYS')

    morningstar_sector = Sector()

    dollar_volume_decile = AverageDollarVolume(window_length=10).deciles()
    top_decile = (dollar_volume_decile.eq(9))

    return Pipeline(
        columns={
            'exchange': exchange,
            'sector_code': morningstar_sector,
            'dollar_volume_decile': dollar_volume_decile
        },
        screen=(nyse_filter & top_decile)
    )

In [8]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result.head(5)

Number of securities that passed the filter: 511


dollar_volume_decile exchange  \
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                      9      NYS   
                          Equity(62 [ABT])                      9      NYS   
                          Equity(64 [ABX])                      9      NYS   
                          Equity(76 [TAP])                      9      NYS   
                          Equity(128 [ADM])                     9      NYS   

                                             sector_code  
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])           101  
                          Equity(62 [ABT])           206  
                          Equity(64 [ABX])           101  
                          Equity(76 [TAP])           205  
                          Equity(128 [ADM])          205

次のレッスンではpiplineから異なるデータセットを扱います。